In [ ]:
import tensorflow as tf
# Use the MNIST dataset of handwritten digits (28x28 grayscale images, 10 classes).
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

In [ ]:

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255     # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Architecture
# • Flatten input (784 features)
# • Dense layer with 64 ReLU units
# • Output layer with 10 units (softmax for TensorFlow)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),              # Input shape: 28x28 pixels
    tf.keras.layers.Flatten(),                          # Flatten to 784 features
    tf.keras.layers.Dense(64, activation='relu'),       # Hidden layer with 64 neurons
    tf.keras.layers.Dense(10, activation='softmax')     # Output layer: 10 classes (digits 0–9)
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',   # Because labels are one-hot encoded
    metrics=['accuracy']
)

# Measure training time
start_time = time.time()
model.fit(x_train, y_train, epochs=5)
end_time = time.time()

print(f"TF Training time: {end_time - start_time:.2f} seconds")

# Evaluate on test data
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=0)
print(f"accuracy : {test_accuracy * 100:.2f}%")
print(f"loss: {test_loss:.4f}")



Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8640 - loss: 0.4900
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9551 - loss: 0.1592
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9673 - loss: 0.1110
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.9749 - loss: 0.0842
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9790 - loss: 0.0699
TF Training time: 24.41 seconds
accuracy : 97.24%
loss: 0.0930


In [ ]:
# Measure inference time and get evaluation metrics
start_inference = time.time()

# model.evaluate runs inference on the test set and returns [loss, accuracy]
test_loss, test_accuracy = model.evaluate(x_test, y_test)

end_inference = time.time()
inference_time = end_inference - start_inference

# Print results
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"Inference Time on Test Set: {inference_time:.2f} seconds")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9676 - loss: 0.1068
Test Accuracy: 97.24%
Inference Time on Test Set: 0.70 seconds


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpxsc0q0l2'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  135388343154640: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135388343155600: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135388343155024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135388343155408: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
# Pytorch Implementation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([transforms.ToTensor(), transforms.Lambda(lambda x: x.view(-1))])
train_loader = DataLoader(datasets.MNIST(root='./data', train=True, transform=transform, download=True), batch_size=32)
test_loader = DataLoader(datasets.MNIST(root='./data', train=False, transform=transform, download=True), batch_size=1000)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 64)    # Fill correct input and output size
        self.fc2 = nn.Linear(64, 10)    # Fill correct input and output size
    def forward(self, x):
        x = F.relu(self.fc1(x))    # Fill correct layer
        return self.fc2(x)    # Fill correct layer

model = Net()
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss()

start = time.time()
for epoch in range(5):
    for x, y in train_loader:
        optimizer.zero_grad()
        pred = model(x)
        loss = loss_fn(pred, y)
        loss.backward()
        optimizer.step()
end = time.time()
print(f"PyTorch Training time: {end - start:.2f} seconds")

PyTorch Training time: 63.81 seconds


In [ ]:
#Inference and Evaluation using PyTorch’s model.eval() + torch.no_grad().
model.eval()
correct = 0

start_time = time.time()

with torch.no_grad():
    for x, y in test_loader:
        output = model(x)
        pred = output.argmax(1)
        correct += (pred == y).sum().item()

end_time = time.time()

inference_time = end_time - start_time

print(f"Test accuracy: {correct / len(test_loader.dataset):.4f}")
print(f"Inference Time: {inference_time:.4f} seconds")

Test accuracy: 0.9647
Inference Time (total): 1.1434 seconds


In [ ]:
# Install ONNX
!pip install onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 101.8 MB/s eta 0:00:00


In [ ]:
dummy_input = torch.randn(1, 784)
torch.onnx.export(model, dummy_input, "model.onnx",
                  input_names=["input"], output_names=["output"])

# TensorFlow custom training loop using tf.GradientTape

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255   # Fill in normalization factor
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

# Prepare datasets
batch_size = 32         # Fill same batch size as in first TF example
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss = loss_fn(y_batch, logits)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))
        train_acc_metric.update_state(y_batch, logits)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3117, Accuracy: 0.2500
Step 100, Loss: 0.4687, Accuracy: 0.7259
Step 200, Loss: 0.3867, Accuracy: 0.8036
Step 300, Loss: 0.3799, Accuracy: 0.8339
Step 400, Loss: 0.2365, Accuracy: 0.8542
Step 500, Loss: 0.3407, Accuracy: 0.8643
Step 600, Loss: 0.4211, Accuracy: 0.8728
Step 700, Loss: 0.1327, Accuracy: 0.8795
Step 800, Loss: 0.1089, Accuracy: 0.8848
Step 900, Loss: 0.2745, Accuracy: 0.8904
Step 1000, Loss: 0.1360, Accuracy: 0.8935
Step 1100, Loss: 0.3220, Accuracy: 0.8969
Step 1200, Loss: 0.3165, Accuracy: 0.9003
Step 1300, Loss: 0.2738, Accuracy: 0.9034
Step 1400, Loss: 0.2884, Accuracy: 0.9055
Step 1500, Loss: 0.1636, Accuracy: 0.9081
Step 1600, Loss: 0.0662, Accuracy: 0.9102
Step 1700, Loss: 0.1036, Accuracy: 0.9128
Step 1800, Loss: 0.1999, Accuracy: 0.9149
Training Accuracy for epoch 1: 0.9164

Epoch 2/5
Step 0, Loss: 0.0633, Accuracy: 1.0000
Step 100, Loss: 0.0747, Accuracy: 0.9570
Step 200, Loss: 0.3089, Accuracy: 0.9520
Step 300, Loss: 0.1288, Accuracy:

## Performance Optimization with Graph Execution using @tf.function

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
import time

# Load and preprocess data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train / 255   # Fill in normalization factor
x_test = x_test / 255   # Fill in normalization factor
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)

# Prepare datasets
batch_size = 32
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)

# Define model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28)),    # Fill size
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),    # Fill number of neurons and activation
    tf.keras.layers.Dense(10, activation='softmax')     # Fill number of neurons and activation
])

# Define loss, optimizer, and metrics
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

@tf.function  # compile the function into a graph
def train_step(x_batch, y_batch):
    with tf.GradientTape() as tape:
        logits = model(x_batch, training=True)
        loss = loss_fn(y_batch, logits)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_acc_metric.update_state(y_batch, logits)
    return loss

# Training loop
epochs = 5
start = time.time()
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(train_dataset):
        loss = train_step(x_batch, y_batch)

        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy():.4f}, Accuracy: {train_acc_metric.result().numpy():.4f}")

    print(f"Training Accuracy for epoch {epoch+1}: {train_acc_metric.result().numpy():.4f}")
    train_acc_metric.reset_state()
end = time.time()
print(f"\nTF Training time: {end - start:.2f} seconds")

# Evaluation loop
for x_batch, y_batch in test_dataset:
    test_logits = model(x_batch, training=False)
    test_acc_metric.update_state(y_batch, test_logits)

print(f"Test Accuracy: {test_acc_metric.result().numpy():.4f}")



Epoch 1/5
Step 0, Loss: 2.3543, Accuracy: 0.1250
Step 100, Loss: 0.4293, Accuracy: 0.7299
Step 200, Loss: 0.2289, Accuracy: 0.8032
Step 300, Loss: 0.2192, Accuracy: 0.8380
Step 400, Loss: 0.4730, Accuracy: 0.8562
Step 500, Loss: 0.2580, Accuracy: 0.8665
Step 600, Loss: 0.2297, Accuracy: 0.8758
Step 700, Loss: 0.2165, Accuracy: 0.8827
Step 800, Loss: 0.2771, Accuracy: 0.8883
Step 900, Loss: 0.2225, Accuracy: 0.8929
Step 1000, Loss: 0.3704, Accuracy: 0.8965
Step 1100, Loss: 0.4104, Accuracy: 0.8988
Step 1200, Loss: 0.1558, Accuracy: 0.9014
Step 1300, Loss: 0.0976, Accuracy: 0.9036
Step 1400, Loss: 0.0955, Accuracy: 0.9062
Step 1500, Loss: 0.5124, Accuracy: 0.9087
Step 1600, Loss: 0.0296, Accuracy: 0.9110
Step 1700, Loss: 0.3256, Accuracy: 0.9132
Step 1800, Loss: 0.0991, Accuracy: 0.9152
Training Accuracy for epoch 1: 0.9164

Epoch 2/5
Step 0, Loss: 0.1072, Accuracy: 1.0000
Step 100, Loss: 0.0315, Accuracy: 0.9511
Step 200, Loss: 0.3384, Accuracy: 0.9541
Step 300, Loss: 0.1710, Accuracy: